**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,Model,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, AveragePooling2D,Reshape,BatchNormalization, Input, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer__:

The function act allows the agent to choose an action in a given state, both during the training period and the test period. During the learning period, the agent have to solve the exploration-exploitation dilemna (trade-off between obtaining new knowledge and using the current knowledge). In fact, the agent have to answer the following question: when he has tried an action that leads to good results, should he continue to choose it or try something else that may lead to better performance? 

Thus, during the training, we want our agent to use the learned policy to test its results and to try other actions to find better behaviour in order to improve the policy (if it exists). To do this, we implement the epsilon-greedy exploration: the agent chooses the action given by the policy with probability $1 - \epsilon$ (exploitation) and try a random action with probability $\epsilon$ (exploration). This allows the trade-off between exploration and exploitation. 

During the test, the agent always chooses the action given by the learned policy to test it.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=5 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer:__

We can see on the function  $\textit{act}$ and $\textit{reset}$ the role of the arrays $\textit{position}$  and $\textit{board}$.

First of all, we can see that the array $\textit{board}$ allows to define where the cheese and poisonous are on the island. So $\textit{board}$ defines the reward we have in the different cells. In fact, $\textit{board}$  represents the environment as a grid where there is $0.5$ for cheese, $0$ for a neutral cell and $-1$ for a poisonous cell. To get the current reward, we just query this array with the command $\textit{board}[x, y]$ where $[x,y]$ is the current position of the agent. Note that, if the rat is on a cell where something is, it eats it and the current cell  $\textit{board}[x, y]$ becomes a neutral cell.

Next, we can see that the array $\textit{position}$ is related to the position of the agent on the grid and the bounds of the grid: we put 1 where the agent is and -1 where the grid boundaries are. We define a boundary around the environment because we consider that a $10 \times 10$ grid is contained in a $14 \times14$ grid to take into account the fact that the rat sees $5\times 5$ cells around it.


Note : i find an error in the code of self.position. There's twice the line $self.position[-2:, :] = -1$ and no occurence of $self.position[:, -2:] = -1$.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """
        Return the action that the agent have to choose when he is in state s.
        Note that a random agent always chooses a random action.
        Input:
            s: the current state.
        Output:
            an action in {0,1,2,3}.
        """
        return  np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        #Reset to a fresh game and get an initial state
        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action, no epsilon-greedy exploration
            action = agent.act(state, train = False)
            
            #Applied the action and get the next state, the reward and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        # No reinforcement strategy

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [14]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random'+str(epochs_test-1)+'.mp4'))

Win/lose count 9.0/7.0. Average score (2.0)
Win/lose count 15.0/8.0. Average score (4.5)
Win/lose count 7.5/13.0. Average score (1.1666666666666667)
Win/lose count 11.0/13.0. Average score (0.375)
Win/lose count 10.5/12.0. Average score (0.0)
Final score: 0.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer:__

   __Part 1:__
    
The $Q$-function is defined by 

\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

We want to isolate the term  $r(s,a)$ so we simply split the sum into two parts

\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \sum_{t = 1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] 
\end{equation*}

By linearity of the expectation, we can write

\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)] + E_{p^{\pi}}[\sum_{t = 1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}

Then we use the markov property to get
\begin{equation*}
\gamma Q^\pi(s,a) = E_{(s',a')\sim p(.|s,a)}[\sum_{t = 1}^{T}\gamma^{t}r(s',a')]
\end{equation*}

and so we get the expression

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

We can see that the $Q_\pi$-function is a fixed point of the Bellman operator $T_\pi$: 

\begin{equation*}
[T_\pi Q](s,a)= r(s,a) + \gamma \sum_{s'\in S} p(s'|s,a) Q(s', \pi(s')) \\
T_\pi Q_\pi = Q_\pi
\end{equation*}

   __Part 2:__
   
The optimal $Q^*$-function maximizes the reward, and it's defined as:

\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

When we use this definition with the previous expression of the  $Q$-function we find 

\begin{equation*}
Q^{*}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{*}(s',a')]
\end{equation*}


Because this $Q^*$-function is optimal, we have $E_{(s',a')\sim p(.|s,a)}[Q^{*}(s',a')] = E_{s'\sim \pi^*(.|s,a)}[ \max_{a'} Q^{*}(s',a')]$ and get

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

The $Q^*$-function is a fixed point of the optimal Bellman operator $T_*$: 

\begin{equation*}
[T_* Q](s,a)= r(s,a) + \gamma \sum_{s'\in S} p(s'|s,a) max_{a' \in A} Q(s', a') \\
T_* Q^* = Q^*
\end{equation*}

__Part 3:__

We want to find the fixed point of the optimal Bellman operator. We can therefore quantify the quality of the current value of the $Q$-function in the sense of this operator by looking at the expected distance between $T_* Q$ and $Q$. We can therefore write:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert [T_* Q](s,a, \theta) -Q(s,a,\theta)\Vert^{2} =E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        """
        Store moves in self.memory.
        Input:
            m: the current move.
        """
        if len(self.memory) < self.max_memory: 
            #if the maximum memory size is not exceeded
            self.memory.append(m)
        else: 
            #if the maximum memory size is exceeded: delete the first move and add the new move
            self.memory[:-1] = self.memory[1:]
            self.memory[-1] = m

    def random_access(self):
        """
        Provide a random access to the memory.
        Output:
            random index of one element of the list self.memory.
        """
        return np.random.randint(len(self.memory))

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        predict_action = self.model.predict(s.reshape(1,5,5,self.n_state))
        return np.argmax(predict_action)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            #Collect self.batch_size informations in the memory.
            index = self.memory.random_access()
            input_states[i] = self.memory.memory[index][0]
            next_state_i = self.memory.memory[index][1]
            action_i = self.memory.memory[index][2]
            reward_i = self.memory.memory[index][3]
            game_over_i = self.memory.memory[index][4]
            
            #Compute the estimation of Q for the curent state
            target_q[i] = self.model.predict(input_states[i].reshape(1,5,5,self.n_state))
            if game_over_i:
                #We don't have a next state
                target_q[i][action_i] = reward_i
            else:
                #Compute the estimation of Q for the next state
                next_target_i = self.model.predict(next_state_i.reshape(1,5,5,self.n_state))
                
                #Update the value of Q for the action we consider with the expression of
                #the previous question
                target_q[i][action_i] = reward_i + self.discount * (np.amax(next_target_i))
             
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.
        bound = 10
        target_q = np.clip(target_q, -bound, bound)

        l = self.model.train_on_batch(input_states, target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # Classical NN Model
        input_layer = Input(shape=(5,5,self.n_state))
        x = Flatten()(input_layer)
        x = Dense(512, activation='relu')(x)
        x = Dense(256, activation='relu')(x)
        x = Dense(64, activation='relu')(x)
        output_layer = Dense(self.n_action, activation='linear')(x)
        model = Model(inputs=input_layer, outputs=output_layer)
        print(model.summary())
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [31]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train'+str((epochs_train-1)-(epochs_train-1)%10)+'.mp4'))

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 5, 5, 2)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               26112     
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_16 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_17 (Dense)             (None, 4)                 260       
Total params: 174,148
Trainable params: 174,148
Non-trainable params: 0
_____________________________________________________

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [11]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        #Classical CNN
        input_layer = Input(shape=(5,5,self.n_state))
        x = Conv2D(256, (2, 2), activation = 'relu')(input_layer)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
        x = Conv2D(256, (2, 2), activation = 'relu')(x)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
        x = Flatten()(x)
        output_layer = Dense(self.n_action, activation='linear')(x)
        model = Model(inputs=input_layer, outputs=output_layer)
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [33]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train'+str((epochs_train-1)-(epochs_train-1)%10)+'.mp4'))

Epoch 000/050 | Loss 0.0047 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/050 | Loss 0.0117 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/050 | Loss 0.0040 | Win/lose count 4.0/5.0 (-1.0)
Epoch 003/050 | Loss 0.0071 | Win/lose count 3.0/2.0 (1.0)
Epoch 004/050 | Loss 0.0141 | Win/lose count 1.5/4.0 (-2.5)
Epoch 005/050 | Loss 0.0175 | Win/lose count 4.0/3.0 (1.0)
Epoch 006/050 | Loss 0.0070 | Win/lose count 4.0/7.0 (-3.0)
Epoch 007/050 | Loss 0.0016 | Win/lose count 3.5/2.0 (1.5)
Epoch 008/050 | Loss 0.0039 | Win/lose count 5.0/5.0 (0.0)
Epoch 009/050 | Loss 0.0078 | Win/lose count 4.0/2.0 (2.0)
Epoch 010/050 | Loss 0.0051 | Win/lose count 4.5/5.0 (-0.5)
Epoch 011/050 | Loss 0.0027 | Win/lose count 4.5/6.0 (-1.5)
Epoch 012/050 | Loss 0.0031 | Win/lose count 5.5/4.0 (1.5)
Epoch 013/050 | Loss 0.0029 | Win/lose count 5.5/3.0 (2.5)
Epoch 014/050 | Loss 0.0169 | Win/lose count 2.0/3.0 (-1.0)
Epoch 015/050 | Loss 0.0061 | Win/lose count 6.0/2.0 (4.0)
Epoch 016/050 | Loss 0.0147 | Win/lose count 6.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [12]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 5, 5, 2)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               26112     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 260       
Total params: 174,148
Trainable params: 174,148
Non-trainable params: 0
_____________________________________________________

In [13]:
#Test different temperatures
temperatures = [0.1,0.3,0.5,0.7,0.9]

for t in temperatures:
    print('\nTemperature : {}'.format(t))
    env = Environment(grid_size=size, max_time=T,temperature=t)
    print('Test of the CNN')
    test(agent_cnn,env, 5, prefix='cnn_test_other_temperature')
    print('Test of the FC')
    test(agent_fc,env, 5, prefix='fc_test_other_temperature')


Temperature : 0.1
Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 0/0. Average score (0.75)
Win/lose count 0/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 2.0/0. Average score (0.8)
Final score: 0.8
Test of the FC
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.0/2.0. Average score (-0.3333333333333333)
Win/lose count 0.5/4.0. Average score (-1.125)
Win/lose count 0.5/2.0. Average score (-1.2)
Final score: -1.2

Temperature : 0.3
Test of the CNN
Win/lose count 2.5/0. Average score (2.5)
Win/lose count 5.0/0. Average score (3.75)
Win/lose count 3.0/0. Average score (3.5)
Win/lose count 7.0/0. Average score (4.375)
Win/lose count 5.0/0. Average score (4.5)
Final score: 4.5
Test of the FC
Win/lose count 2.5/3.0. Average score (-0.5)
Win/lose count 1.5/1.0. Average score (0.0)
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.5/2.0.

In [35]:
HTML(display_videos('cnn_test'+str(epochs_test-1)+'.mp4'))

In [36]:
HTML(display_videos('fc_test'+str(epochs_test-1)+'.mp4'))

__Answer:__

It seems that the agent doesn't explore the environment enough. Once he has eaten a piece of cheese or when he has eaten nothing, he prefers not to play in order not to diminish his rewards by eating poison. He then oscillates between two empty cells and stays in a local minimum. The network ignores the rest of the map and doesn't take into account the already explored part of this one. 

The temperature is used to regulate the amount of cheese on the board: for a high temperature value there is more cheese in the environment and the agent has no problem to win; for a low temperature value there is less cheese on the board and the agent tends not to explore the environment enough to be able to win.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [17]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            agent.set_epsilon(agent.epsilon*0.9)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end. Train = True to take into account the exploration.
            prev_state = state
            state, reward, game_over = env.act(action, train = True) 

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        #new state that describes if a cell has been visited or not
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        #If train, we take into account the malus of exploration
        if train:
            reward = -self.malus_position[self.x, self.y]
        #We increase the malus to force faster exploration
        self.malus_position[self.x, self.y]+= 0.1
        
        #If train we can change the impact of the rewards
        if train:
            reward = reward + 10*self.board[self.x, self.y]
        else:
            reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [18]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore'+str((epochs_train-1)-(epochs_train-1)%10)+'.mp4'))

Epoch 000/050 | Loss 0.5688 | Win/lose count 65.0/140.99999999999991 (-75.99999999999991)
Epoch 001/050 | Loss 0.1519 | Win/lose count 50.0/153.0 (-103.0)
Epoch 002/050 | Loss 0.1838 | Win/lose count 90.0/93.7 (-3.700000000000003)
Epoch 003/050 | Loss 0.1630 | Win/lose count 80.0/93.49999999999997 (-13.499999999999972)
Epoch 004/050 | Loss 0.2667 | Win/lose count 90.0/95.29999999999998 (-5.299999999999983)
Epoch 005/050 | Loss 0.5109 | Win/lose count 130.0/65.30000000000005 (64.69999999999995)
Epoch 006/050 | Loss 0.1395 | Win/lose count 215.0/68.20000000000007 (146.79999999999993)
Epoch 007/050 | Loss 0.3465 | Win/lose count 135.0/72.90000000000005 (62.09999999999995)
Epoch 008/050 | Loss 0.1002 | Win/lose count 155.0/72.29999999999998 (82.70000000000002)
Epoch 009/050 | Loss 0.0719 | Win/lose count 170.0/51.20000000000006 (118.79999999999994)
Epoch 010/050 | Loss 0.4159 | Win/lose count 165.0/47.30000000000003 (117.69999999999996)
Epoch 011/050 | Loss 0.1549 | Win/lose count 90.0/143

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore'+str(epochs_test-1)+'.mp4'))

Win/lose count 23.5/2.0. Average score (21.5)
Win/lose count 24.0/3.0. Average score (21.25)
Win/lose count 28.5/0. Average score (23.666666666666668)
Win/lose count 20.5/2.0. Average score (22.375)
Win/lose count 26.0/0. Average score (23.1)
Final score: 23.1


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [24]:
#First step: save state and action of a game

def generate_games(trained_agent,env,epoch,prefix='mimick'):
    # Number of won games
    score = 0
    loss = 0
    
    states = []
    actions = []

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
        # The agent performs an action
            action = agent.act(state, train = False)
            #Save the state and the action (one_hot format)
            states.append(state)
            one_hot_action = np.zeros(4)
            one_hot_action[action]=1
            actions.append(one_hot_action)
            
            prev_state = state
            state, reward, game_over = env.act(action)

        # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epoch))
    
    return np.array(states), np.array(actions)

states, actions = generate_games(agent, env, 5)

Win/lose count 19.5/0. Average score (19.5)
Win/lose count 21.0/2.0. Average score (19.25)
Win/lose count 21.5/0. Average score (20.0)
Win/lose count 21.0/2.0. Average score (19.75)
Win/lose count 30.5/1.0. Average score (21.7)
Final score: 21.7


In [25]:
#Second step: create a network to learn the association between a state and an action
#in a wining game

class DQN_mimick(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_mimick, self).__init__(*args,**kwargs)

        input_layer = Input(shape=(5,5,self.n_state))
        x = Conv2D(256, (2, 2), activation = 'relu')(input_layer)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
        x = Conv2D(256, (2, 2), activation = 'relu')(x)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
        x = Flatten()(x)
        x = Dense(64)(x)
        output_layer = Dense(self.n_action, activation='linear')(x)
        model = Model(inputs=input_layer, outputs=output_layer)
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
    
    def train(self, data, label):
        l = self.model.train_on_batch(data, label)
        return l
        
agent_mimick = DQN_mimick(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)

In [27]:
#Third step: train the network and evaluate the performance

def train_mimick(agent,data,label,epoch,prefix='mimick_'):
    for e in range(epoch):
        
        loss = 0
        idx = np.arange(data.shape[0])
        np.random.shuffle(idx)
        
        #Create the batch and train the model on it
        batch_size = 32
        for batch_idx in range(int(data.shape[0]/batch_size)+1):
            batch = idx[batch_idx*batch_size:min((batch_idx+1)*batch_size,data.shape[0])]
            batch_data = data[batch,:,:,:]
            batch_gt = label[batch]
            loss += agent.train(batch_data,batch_gt)
        
        print("Epoch {}/{} | Loss {}".format(e, epoch, loss))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

train_mimick(agent_mimick, states, actions, 100)

Epoch 0/100 | Loss 5.950964063405991
Epoch 1/100 | Loss 5.4331530928611755
Epoch 2/100 | Loss 5.191505238413811
Epoch 3/100 | Loss 4.97911362349987
Epoch 4/100 | Loss 4.8087047934532166
Epoch 5/100 | Loss 4.666437983512878
Epoch 6/100 | Loss 4.513818606734276
Epoch 7/100 | Loss 4.448625296354294
Epoch 8/100 | Loss 4.373541176319122
Epoch 9/100 | Loss 4.244263432919979
Epoch 10/100 | Loss 4.153473623096943
Epoch 11/100 | Loss 3.997155539691448
Epoch 12/100 | Loss 4.009425662457943
Epoch 13/100 | Loss 3.9680618792772293
Epoch 14/100 | Loss 3.8887380361557007
Epoch 15/100 | Loss 3.7833029106259346
Epoch 16/100 | Loss 3.7553384378552437
Epoch 17/100 | Loss 3.620341271162033
Epoch 18/100 | Loss 3.575981117784977
Epoch 19/100 | Loss 3.547938324511051
Epoch 20/100 | Loss 3.463156998157501
Epoch 21/100 | Loss 3.454955294728279
Epoch 22/100 | Loss 3.373303048312664
Epoch 23/100 | Loss 3.3368071988224983
Epoch 24/100 | Loss 3.281106159090996
Epoch 25/100 | Loss 3.224342867732048
Epoch 26/100 | L

In [28]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test(agent_mimick,env,epochs_test,prefix='mimick_test_explore')
HTML(display_videos('mimick_test_explore0.mp4'))

Win/lose count 26.0/5.0. Average score (21.0)
Win/lose count 20.5/4.0. Average score (18.75)
Win/lose count 12.5/4.0. Average score (15.333333333333334)
Win/lose count 18.0/5.0. Average score (14.75)
Win/lose count 19.0/8.0. Average score (14.0)
Final score: 14.0


__Answer__:

In a first step we created several state-action pairs that are used in a winning game. These pairs are created using the DQN model proposed in the previous section. We then train a network that from a state should predict the corresponding action.

It is possible to see that this network has good performances. However, it remains a little less good than the results obtained with the DQN. This may be due to the training time of the mimik network. 

***